In [2]:
import numpy as np
import pickle
import logging
np.random.seed(1337)
import re
import codecs
import jieba
import math
import numpy as np
from itertools import product, count  
from heapq import nlargest  
from gensim.models import word2vec  
from sklearn.utils import shuffle
try:
    reload(sys)
    sys.setdefaultencoding('utf-8')
except:
    pass

## 打开正面和负面的新闻
格式是 {标题：新闻内容}

In [3]:
import json
with open("pos.JSON", 'r') as fp:
    pos_text = json.load( fp)
with open("neg.JSON", 'r') as fp:
    neg_text = json.load( fp)

In [4]:
#分句
def sentence_split(str_centence):
    list_ret = list()
    for s_str in str_centence.split('。'):
        if '?' in s_str:
            list_ret.extend(s_str.split('？'))
        elif '!' in s_str:
            list_ret.extend(s_str.split('！'))
        else:
            list_ret.append(s_str.strip())
    return list_ret

In [5]:
#建立停词
stopwordset = set()
with open('sa/jieba_dict/stopwords.txt','r',encoding = 'utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))

#分词wrapper
def seg_art_list(art_list):
    corpus = []
    for title, art in art_list.items():
        corpus.append(" ".join(seg_sent(art)))
    return corpus

#分词，去标点符号，去停词，去英文，去数字
#art： 单篇文章
#return：所有的单词
output = open('word_seg.txt','w')
def seg_sent(art):
    r = '[’!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]+'  
    art = art.strip()
    art = re.sub(r, '', art)
    art = re.sub("[a-z]","",art)
    art = re.sub("[0-9]","",art)
    l = []

    seg_list = list(jieba.cut(art, cut_all = False))
    for word in seg_list:
        if word not in stopwordset and word != ' ' and word != "\n" and word != "\n\n":
                output.write(word + ' ')
                l.append(word)
    return l


In [6]:
#建立和保存正面新闻的 分词
#建立和保存负面新闻的 分词

pos_tfidf = seg_art_list(pos_text)
neg_tfidf = seg_art_list(neg_text)
with open("pos_tfidf.JSON", 'w') as fp:
    json.dump(pos_tfidf, fp)
with open("neg_tfidf.JSON", 'w') as fp:
    json.dump(neg_tfidf, fp)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/zy/lygrnf2j60555xlt5007s4lm0000gp/T/jieba.cache
Loading model cost 0.769 seconds.
Prefix dict has been built succesfully.


## 建模

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.metrics.scorer import make_scorer
from sklearn import linear_model
from sklearn import metrics
import operator
from time import time
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report as clsr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cross_validation import train_test_split as tts
import sklearn
import timeit
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

In [8]:
#Y： 1为正面，-1为负面
X = list(pos_tfidf) + list(neg_tfidf)
y = [1] * len(pos_tfidf) + [-1]*len(neg_tfidf)

In [9]:
len(y)

925

In [12]:

cv = CountVectorizer(ngram_range=(1,2), min_df=20,max_df = 0.9,lowercase=False)
_X = cv.fit_transform(X).toarray()
tfidf = TfidfVectorizer()
_X = tfidf.fit_transform(X)
feature_names = list(tfidf.get_feature_names())
classifier= RandomForestClassifier(n_estimators=100,random_state=5,min_samples_leaf=2)



In [13]:
X_train, X_test, y_train, y_test = tts(_X, y, test_size=0.2)

model = classifier.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("classifier result:")
print(clsr(y_test, y_pred, target_names=['neg', 'pos']))

print("confusion matrix")
print(metrics.confusion_matrix(y_test, y_pred))

classifier result:
             precision    recall  f1-score   support

        neg       0.98      0.68      0.80        63
        pos       0.86      0.99      0.92       122

avg / total       0.90      0.89      0.88       185

confusion matrix
[[ 43  20]
 [  1 121]]


In [16]:
feature_importance = pd.DataFrame({'name':feature_names, 'importance':model.feature_importances_},columns = ['name','importance'])

In [17]:
sort_importance = feature_importance.sort_values(by= ['importance'],ascending = False)
sort_importance.head(50)

,name,importance
2060,买入,0.068571
1724,中性,0.051280
5513,同比,0.012709
1323,下降,0.012206
11027,毛利率,0.011635
9458,收入,0.009859
1951,主要,0.008732
3899,减少,0.008249
10305,有望,0.007404
252,eps,0.007348


In [18]:
sort_importance.to_csv("sort_importance.csv",index=False)